## Описание
Берёт filename.txt

- 1. AUTHOR_2021@textname_1
- bu	on'ariwun	ikənŋen...	-	festival'du (первая строка после id строки)
- bu	on'a-ri-wun	ikən-ŋe-n	festival'-du (вторая строка)
- Мы	писать-PST-1PL	песня-DST-POSS.3SG	фестиваль-DAT (третья строка)
- ‘Мы написали песню для фестиваля.’ (четвёртая строка)
- ...

Дальше может быть что угодно, строка id вычисляется по re.match(r"\d+\..*$", line).

Берёт meta из первой id строки.

Записывает json файл в [формате tsakorpus](https://tsakorpus.readthedocs.io/en/latest/data_model.html).


In [1]:
import json
import pprint
import re
from itertools import zip_longest

In [17]:
def get_meta(text, filename):
    pattern = ' (.+?)_([0-9]+?)@(.+?)_([0-9]+)'
    result = re.search(pattern, text)
    filename = filename
    meta = {"filename": filename,
        "author": result.group(1),
        "year": result.group(2),
        "recplace": "Bystraja district",
        "title": result.group(3),
        "genre": "newspaper"}
        # "genre": "tale"}
    return meta

In [8]:
PUNCTUATION = '!"#$%&\'()*+, -—./:;<=>?@[\]^_`{|}~'

In [15]:
filenames = ["089_199702_1_eve_annotated", "089_199702_2_eve_annotated"]

In [16]:
for filename in filenames:
    eve_data = open(filename+'.txt').read()
    sentences = []
    sentences_ru = []
    lines = eve_data.splitlines()
    for line_id, line in enumerate(lines):
        if re.match(r"\d+\..*$", line):
            id = int(re.search(r"\d+", line).group())  # id в тексте
            if id == 1:
                 meta = get_meta(line, filename)  # meta текста
            text = lines[line_id+1]
            words = text.split()  # слова в предложении
            word_borders = []
            word_borders.append(0)
            for sym_id, symbol in enumerate(list(lines[line_id+1])):
                if symbol.isspace():
                    word_borders.append(sym_id-1)
                    word_borders.append(sym_id+1)
            word_borders.append(len(lines[line_id+1])-1)
            chunks = list(x for x in zip_longest(*[iter(word_borders)] * 2))  # границы слов в предложении
            parts_list = lines[line_id+2].split()  # список с морфемами предложения
            gloss_list = lines[line_id+3].split()  # список с глоссами предложения
        
            gloss_index_list = []
            for gl_parts in list(zip(gloss_list, parts_list)):
                gloss_index = ""
                for para in list(zip(gl_parts[0].split("-"), gl_parts[1].split("-"))):
                    gloss_index += para[0]+"{"+ para[1] + "}-"
                gloss_index_list.append(gloss_index)  # список с gloss_index предложения

            for word_id, word in enumerate(words):
                if word in PUNCTUATION:
                    words.remove(word)
                    chunks.pop(word_id)

            words_info = []
            for word_id, word in enumerate(words):
                word_info = {'wf': word,
                             'off_start': chunks[word_id][0],
                             'off_end': chunks[word_id][1]+1,
                             'wtype': 'word' if word not in PUNCTUATION else 'punct',  # что с тире делать
                             'ana': [{'parts': parts_list[word_id],
                                      'gloss': gloss_list[word_id],
                                      'gloss_index': gloss_index_list[word_id]}],
                              'next_word': word_id+1,
                              'sentence_index': word_id,
                              'sentence_index_neg': len(words)-word_id}
                words_info.append(word_info)
            sentence = {'text': text, 'words': words_info,
                        "lang": 0, "para_alignment": [{"off_start": 0,
                                                       "off_end": len(text),
                                                       "para_id": id}]}
            sentences.append(sentence)

            text_ru = lines[line_id+4]
            if text_ru != '':
                words_ru = text_ru.split()  # слова в предложении
                word_ru_borders = [0]
                for sym_id, symbol in enumerate(list(lines[line_id+4])):
                    if symbol.isspace():
                        word_ru_borders.append(sym_id-1)
                        word_ru_borders.append(sym_id+1)
                word_ru_borders.append(len(lines[line_id+4])-1)
                chunks = list(x for x in zip_longest(*[iter(word_ru_borders)] * 2))  # границы слов в предложении
                words_ru_info = []
                for word_ru_id, word_ru in enumerate(words_ru):
                    word_ru_info = {'wf': word_ru,
                                 'off_start': chunks[word_ru_id][0],
                                 'off_end': chunks[word_ru_id][1]+1,
                                 'wtype': 'word' if word_ru not in PUNCTUATION else 'punct',
                                 'next_word': word_ru_id+1,
                                 'sentence_index': word_ru_id,
                                 'sentence_index_neg': len(words_ru)-word_ru_id}
                    words_ru_info.append(word_ru_info)
                sentence_ru = {"text": text_ru, "words": words_ru_info,
                               "lang": 1, "para_alignment": [{"off_start": 0,
                                                              "off_end": len(text_ru),
                                                              "para_id": id}]}
                sentences_ru.append(sentence_ru)
    sentences.extend(sentences_ru)
# Что со строкой комментариев? Её нет.
    data = {'meta': meta,
            'sentences': sentences}
    jsonString = json.dumps(data, ensure_ascii=False)
    with open(filename + '.json', 'w', encoding='utf-8') as f:
        f.write(jsonString)

In [ ]:
# data